# Import data
use python to downlaod and load statscanada odhf dataset into catalogs, schema and unique table specified below 


make sur eyou change catalog and schema name to workshop specific catalog and schema and make table name unique tp your self

In [0]:
# Define variables for catalog name, schema, and table name
catalog_name = "razi_demo"
schema_name = "vch_workshop"
table_name = "raw_odhf_razi_data"
cleaned_table_name = "clean_odhf_razi_data"


In [0]:
# Step 1: Download and Load statscanada odhf dataset

import pandas as pd
import zipfile
import requests
from io import BytesIO

url = "https://www150.statcan.gc.ca/n1/en/pub/13-26-0001/2020001/ODHF_v1.1.zip"
response = requests.get(url)
with zipfile.ZipFile(BytesIO(response.content)) as z:
    with z.open('ODHF_v1.1/odhf_v1.1.csv') as f:
        raw_df = pd.read_csv(f,encoding='ISO-8859-1')

display(raw_df)

In [0]:
#step 2:  convert df to a Spark DataFrame first and then write the Spark DataFrame to a table
 
spark_df = spark.createDataFrame(raw_df)

spark_df.write.mode("overwrite").saveAsTable(f"{catalog_name}.{schema_name}.{table_name}")

# Data enrichment

using teh data profile here we realized 6.88% of rows miss lattitude and longitude, let's drop them 

In [0]:
df_cleaned = raw_df.dropna(subset=["latitude", "longitude"])
display(df_cleaned)
# use data profile to confirm the lat/long are valid now

In [0]:
df_cleaned_spark = spark.createDataFrame(df_cleaned)
df_cleaned_spark.write.mode("overwrite").saveAsTable(f"{catalog_name}.{schema_name}.{cleaned_table_name}")

display(df_cleaned_spark)

# Create visualization

visualize the facilities on map

In [0]:
%pip install folium

import folium
from folium.plugins import MarkerCluster



In [0]:
# Ensure df_cleaned is a Spark DataFrame
df_cleaned = spark.createDataFrame(df_cleaned)

# Convert Spark DataFrame to Pandas DataFrame
df_pandas = df_cleaned.toPandas()

# Create a map centered around the average latitude and longitude
map_center = [df_pandas['latitude'].mean(), df_pandas['longitude'].mean()]
facility_map = folium.Map(location=map_center, zoom_start=10)

# Add a marker cluster to the map
marker_cluster = MarkerCluster().add_to(facility_map)

# Add markers to the cluster
for idx, row in df_pandas.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['facility_name']
    ).add_to(marker_cluster)

# Display the map
display(facility_map)